# 14. AUTO DATA: OVERFITTING
---

## 1. Introducing the Data

**Note**: The `mpg` column is our target column and is the one we want to predict using the other features.

In [12]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 99)
pd.set_option("display.max_rows", 999)
pd.set_option('precision', 3)

cols = ["mpg", "cylinders", "displacement", "horsepower", "weight", 
        "acceleration", "model year", "origin", "car name"]
cars = pd.read_csv('data/auto_mpg.data',
                    delim_whitespace=True,
                    names=cols)
filtered_cars = cars[cars['horsepower'] != '?'].copy()
filtered_cars['horsepower'] = filtered_cars['horsepower'].astype('float')
print(filtered_cars.shape)
filtered_cars.head()

(392, 9)


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


## 2. Bias-Variance Tradeoff


In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def train_and_test(cols):
    # Split into features & target.
    features = filtered_cars[cols]
    target = filtered_cars["mpg"]
    # Fit model.
    lr = LinearRegression()
    lr.fit(features, target)
    # Make predictions on training set.
    predictions = lr.predict(features)
    # Compute MSE and Variance.
    mse = mean_squared_error(filtered_cars["mpg"], predictions)
    variance = np.var(predictions)
    return(mse, variance)
    
cyl_mse, cyl_var = train_and_test(["cylinders"])
weight_mse, weight_var = train_and_test(["weight"])
print(cyl_mse, cyl_var)
print(weight_mse, weight_var)

24.02017956815553 36.74255887416017
18.6766165974193 42.08612184489641


## 3. Multivariate Models

In [15]:
two_mse, two_var = train_and_test(
    ["cylinders", "displacement"])
three_mse, three_var = train_and_test(
    ["cylinders", "displacement", "horsepower"])
four_mse, four_var = train_and_test(
    ["cylinders", "displacement", "horsepower", "weight"])
five_mse, five_var = train_and_test(
    ["cylinders", "displacement", "horsepower", "weight", 
     "acceleration"])
six_mse, six_var = train_and_test(
    ["cylinders", "displacement", "horsepower", "weight", 
     "acceleration", "model year"])
seven_mse, seven_var = train_and_test(
    ["cylinders", "displacement", "horsepower", "weight", 
     "acceleration","model year", "origin"])

print(two_mse, two_var)
print(three_mse, three_var)
print(four_mse, four_var)
print(five_mse, five_var)
print(six_mse, six_var)
print(seven_mse, seven_var)

21.282057055586364 39.480681386729316
20.252954839714228 40.509783602601445
17.763860571843846 42.99887787047185
17.76139610540622 43.001342336909396
11.590170981415227 49.172567460900346
10.847480945000454 49.915257497315146


## 4. Cross-Validation
- A good way to detect if your model is overfitting is to compare the `in-sample error` and the `out-of-sample error`, or the training error with the test error. So far, we calculated the in sample error by testing the model over the same data it was trained on. To calculate the out-of-sample error, we need to test the data on a test set of data. We unfortunately don't have a separate test dataset and we'll instead use cross validation.
- If a model's cross validation error (out-of-sample error) is much higher than the in sample error, then your data science senses should start to tingle. This is the first line of defense against overfitting and is a clear indicator that the trained model doesn't generalize well outside of the training set.
- Let's create a new function to handle performing the cross validation and computing the cross validation error.

In [17]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def train_and_cross_val(cols):
    features = filtered_cars[cols]
    target = filtered_cars["mpg"]
    
    variance_values = []
    mse_values = []
    
    kf = KFold(n_splits=10, shuffle=True, random_state=3)    
    
    for train_index, test_index in kf.split(features):        
        X_train, X_test = features.iloc[train_index], features.iloc[test_index]
        y_train, y_test = target.iloc[train_index], target.iloc[test_index]        
        
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        predictions = lr.predict(X_test)        
        
        mse = mean_squared_error(y_test, predictions)
        var = np.var(predictions)        
        variance_values.append(var)
        mse_values.append(mse)   
    
    avg_mse = np.mean(mse_values)
    avg_var = np.mean(variance_values)
    return(avg_mse, avg_var)
        
two_mse1, two_var1 = train_and_cross_val(
    ["cylinders", "displacement"])
three_mse1, three_var1 = train_and_cross_val(
    ["cylinders", "displacement", "horsepower"])
four_mse1, four_var1 = train_and_cross_val(
    ["cylinders", "displacement", "horsepower", "weight"])
five_mse1, five_var1 = train_and_cross_val(
    ["cylinders", "displacement", "horsepower", "weight", 
     "acceleration"])
six_mse1, six_var1 = train_and_cross_val(
    ["cylinders", "displacement", "horsepower", "weight", 
     "acceleration", "model year"])
seven_mse1, seven_var1 = train_and_cross_val(
    ["cylinders", "displacement", "horsepower", "weight", 
     "acceleration","model year", "origin"])

print(two_mse1, two_var1)
print(three_mse1, three_var1)
print(four_mse1, four_var1)
print(five_mse1, five_var1)
print(six_mse1, six_var1)
print(seven_mse1, seven_var1)

21.584370274954374 38.902525313756016
20.655622193882955 40.091287956606934
18.169683239081884 42.50764364364439
18.283038517172052 42.59873630014678
12.099685425467118 48.92824696771803
11.418131971812054 49.90431373098729


During cross validation, the more features we added to the model, the lower the mean squared error got. This is a good sign and indicates that the model generalizes well to new data it wasn't trained on. As the mean squared error value went down, however, the variance of the predictions went up. This is to be expected, since the models with lower squared error values had higher model complexity, which tends to be more sensitive to small variations in input values (or high variance).

For each model, let's plot the error and variance to get a better idea of the tradeoff as the number of features increased.